In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
import tensorflow_probability as tfp


In [5]:
# Loading the dataset
co2 = sm.datasets.co2.load_pandas()
df_co2 = co2.data

df_co2.fillna(df_co2.bfill(), inplace = True)

In [6]:
trend = tfp.sts.LocalLinearTrend(observed_time_series=df_co2)

Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.


In [8]:
seasonal = tfp.sts.Seasonal(num_seasons=52, observed_time_series=df_co2)

Instructions for updating:
Do not call `graph_parents`.
Instructions for updating:
`MultivariateNormalFullCovariance` is deprecated, use `MultivariateNormalTriL(loc=loc, scale_tril=tf.linalg.cholesky(covariance_matrix))` instead.


In [9]:
model = tfp.sts.Sum([trend, seasonal], observed_time_series=df_co2)